In [ ]:
from process import Process

p1 = Process(1, 8)
p2 = Process(2, 2)
p3 = Process(3, 7)
p4 = Process(4, 3)
p5 = Process(5, 5)

process_list = [p1, p2, p3, p4, p5]

In [ ]:
import pandas as pd
from scheduler import *

scheduler(process_list)

data = {
    "Process": cpu_scheduling_pname,
    "Start": scheduling_starts,
    "End": scheduling_ends,
}

df = pd.DataFrame(data)
df["Quanta"] = df["End"] - df["Start"]
df["Color"] = colors

df

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("ggplot")

fig = plt.figure(num=1)
plt.xlabel("Quanta time")
plt.ylabel("Process")
plt.barh(y=df["Process"], left=df["Start"], width=df["Quanta"], color=df["Color"])
plt.show()